In [1]:
import os
import sys

MODULE_PATH = 'C:\Github\DL_Study\Base'

sys.path.insert(0, MODULE_PATH)

In [2]:
# import
import numpy
from config import *
from optim import Adam
from models import GruModelReg

In [3]:
# configuration setting
def model_config():
    # parameter for LSTM Model
    epochs = [30]
    batch_size = [64]
    learning_rate = [0.01, 0.001]
    
    # create config data
    configs = []
    for i in epochs:
        for j in batch_size:
            for k in learning_rate:
                config = [i, j, k]
                configs.append(config)
    return configs

# fucntion for fit cnn model using configs
def model_fit(train_X, train_y, config):
    # unpack config
    n_epochs, n_batch, learning_rate = config
    model = GruModelReg(time_size=24, hidden_size=64, feature_size=7)
    # fit model and return
    model.fit(train_X=train_X, train_y=train_y, epochs=n_epochs, 
              batch_size=n_batch, learning_rate=learning_rate)
    return model

def MAE_metric(x, t):
    return np.mean(numpy.abs(x-t))

def MSE_metric(x, t):
    return np.mean((x-t)**2)

In [4]:
# testcell
import pandas as pd
import numpy
import time
from datetime import datetime

np.random.seed(42)
numpy.random.seed(42)

data_url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/00235/household_power_consumption.zip'
df_parser = lambda x: datetime.strptime(x, '%d/%m/%Y %H:%M:%S')
df = pd.read_csv(data_url, compression='zip', sep=';', parse_dates=[['Date', 'Time']], date_parser=df_parser, index_col=0)
df.replace('?', numpy.nan, inplace=True)    # nan data
df = df['2007']
df = df[:10000]
df = df.astype(float)
df.head()

C:\Users\lab\anaconda3\envs\pytorch\lib\site-packages\IPython\core\interactiveshell.py:3169: DtypeWarning: Columns (2,3,4,5,6,7) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
C:\Users\lab\AppData\Local\Temp/ipykernel_14560/2943368522.py:14: FutureWarning: Indexing a DataFrame with a datetimelike index using a single string to slice the rows, like `frame[string]`, is deprecated and will be removed in a future version. Use `frame.loc[string]` instead.
  df = df['2007']


,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3
Date_Time,,,,,,,
2007-01-01 00:00:00,2.580,0.136,241.97,10.6,0.0,0.0,0.0
2007-01-01 00:01:00,2.552,0.100,241.75,10.4,0.0,0.0,0.0
2007-01-01 00:02:00,2.550,0.100,241.64,10.4,0.0,0.0,0.0
2007-01-01 00:03:00,2.550,0.100,241.71,10.4,0.0,0.0,0.0
2007-01-01 00:04:00,2.554,0.100,241.98,10.4,0.0,0.0,0.0


In [5]:
# series data to img function
def series_to_img(dataset, time_step=1):
    num = dataset.shape[1]      # features num
    df = pd.DataFrame(dataset)
    cols, names = list(), list()
    # sequence t-n to t-1
    for i in range(time_step, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(num)]

    for i in range(0, 1):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(num)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(num)]

    agg = pd.concat(cols, axis=1)
    agg.columns = names
    agg.dropna(inplace=True)
    return agg

from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import MinMaxScaler

df.ffill(axis=1, inplace=True)
dataset = df.values
dataset = dataset.astype('float')

n_inputs = 24
n_features = 7
del_idx = n_inputs * n_features + 1
del_cols = [i for i in range(del_idx, del_idx+n_features-1)]
new_df = series_to_img(dataset, n_inputs)
new_df.drop(new_df.columns[del_cols], axis=1, inplace=True)
new_df.head()

,var1(t-24),var2(t-24),var3(t-24),var4(t-24),var5(t-24),var6(t-24),var7(t-24),var1(t-23),var2(t-23),var3(t-23),...,var6(t-2),var7(t-2),var1(t-1),var2(t-1),var3(t-1),var4(t-1),var5(t-1),var6(t-1),var7(t-1),var1(t)
24,2.580,0.136,241.97,10.6,0.0,0.0,0.0,2.552,0.1,241.75,...,1.0,0.0,2.650,0.218,241.67,11.0,0.0,2.0,0.0,2.682
25,2.552,0.100,241.75,10.4,0.0,0.0,0.0,2.550,0.1,241.64,...,2.0,0.0,2.682,0.258,242.45,11.0,0.0,1.0,0.0,2.660
26,2.550,0.100,241.64,10.4,0.0,0.0,0.0,2.550,0.1,241.71,...,1.0,0.0,2.660,0.252,241.60,11.0,0.0,1.0,0.0,2.650
27,2.550,0.100,241.71,10.4,0.0,0.0,0.0,2.554,0.1,241.98,...,1.0,0.0,2.650,0.250,241.14,11.0,0.0,2.0,0.0,2.654
28,2.554,0.100,241.98,10.4,0.0,0.0,0.0,2.550,0.1,241.83,...,2.0,0.0,2.654,0.250,241.38,11.0,0.0,1.0,0.0,2.642


In [6]:
n_splits = 10
train_test_split = TimeSeriesSplit(n_splits=n_splits+1, gap=n_inputs).split(new_df)
next(train_test_split)

configs = model_config()
history = []

best_rmse, best_mse, best_mae = [], [], []
learning_time = []
i = 1

print('config : epochs, batch_size, learning_rate')

# neted cross validation
for train_cv_indices, test_cv_indices in train_test_split:
    print(f'fold : {i}/{n_splits}')
    i+=1

    # split x, y data
    train_cv_X, train_cv_y = new_df.iloc[train_cv_indices, :-1].values, new_df.iloc[train_cv_indices,-1].values
    test_cv_X, test_cv_y = new_df.iloc[test_cv_indices, :-1].values, new_df.iloc[test_cv_indices, -1].values

    # length for validation set
    test_length = int(len(train_cv_X)*0.2)

    # scaling data
    scaler_x = MinMaxScaler()
    train_cv_X = scaler_x.fit_transform(train_cv_X)
    test_cv_X = scaler_x.transform(test_cv_X)

    train_X, val_X = train_cv_X[:-test_length, :], train_cv_X[-test_length:, :]
    train_y, val_y = train_cv_y[:-test_length], train_cv_y[-test_length:]

    # reshape
    # inner loop
    train_X = train_X.reshape(-1,  n_inputs, n_features)
    val_X = val_X.reshape(-1, n_inputs, n_features)
    train_y = train_y.reshape(-1, 1)
    val_y = val_y.reshape(-1, 1)

    # outer loop
    train_cv_X = train_cv_X.reshape(-1,  n_inputs, n_features)
    test_cv_X = test_cv_X.reshape(-1, n_inputs, n_features)
    train_cv_y = train_cv_y.reshape(-1, 1)
    test_cv_y = test_cv_y.reshape(-1, 1)

    # model fit, inner
    errors = []
    for idx, cfg in enumerate(configs):
        print(f' == train {cfg} model == ', end=' ')
        model = model_fit(train_X, train_y, cfg)
        model.reset_state()
        predicted = model.predict(val_X)
        if GPU:
            predicted = np.asnumpy(predicted)
        error = np.sqrt(MSE_metric(predicted, val_y))   # rmse
        print(f' error(RMSE):{error}')
        if errors:
            if error < min(errors):
                param = idx
        else:
            param = idx
        errors.append(error)

    history.append(errors)
    
    # check start time
    start_time = time.time()
    # model fitting
    selected_model = model_fit(train_cv_X,train_cv_y, configs[param])
    # check duration
    duration = time.time()-start_time
    selected_model.reset_state()
    predicted = selected_model.predict(test_cv_X)
    if GPU:
        predicted = np.asnumpy(predicted)

    rmse = np.sqrt(MSE_metric(predicted, test_cv_y))
    mse = MSE_metric(predicted, test_cv_y)
    mae = MAE_metric(predicted, test_cv_y)
    best_rmse.append(rmse)
    best_mse.append(mse)
    best_mae.append(mae)
    learning_time.append(duration)

    # model eval
    print(f'train-size:{train_X.shape[0]}, val-size:{val_X.shape[0]}, test-size:{test_cv_X.shape[0]}')
    print(f'best_model => error(rmse) : {rmse.item():.2f}, param:{configs[param]}, times : {duration:.3f}')
    print()

config : epochs, batch_size, learning_rate
fold : 1/10
 == train [30, 64, 0.01] model ==   error(RMSE):1.1333458675546715
 == train [30, 64, 0.001] model ==   error(RMSE):0.13851851520822775
train-size:1314, val-size:328, test-size:831
best_model => error(rmse) : 0.15, param:[30, 64, 0.001], times : 71.306

fold : 2/10
 == train [30, 64, 0.01] model ==   error(RMSE):0.9365959471200035
 == train [30, 64, 0.001] model ==   error(RMSE):1.4617436214724755
train-size:1979, val-size:494, test-size:831
best_model => error(rmse) : 1.26, param:[30, 64, 0.01], times : 109.386

fold : 3/10
 == train [30, 64, 0.01] model ==   error(RMSE):2.8709233813198582
 == train [30, 64, 0.001] model ==   error(RMSE):0.1308936007783155
train-size:2644, val-size:660, test-size:831
best_model => error(rmse) : 0.19, param:[30, 64, 0.001], times : 145.805

fold : 4/10
 == train [30, 64, 0.01] model ==   error(RMSE):8.221698166526437
 == train [30, 64, 0.001] model ==   error(RMSE):0.1183228363936421
train-size:330

In [7]:
def model_evaluation(mse, rmse, mae):
    mse = np.array(mse)
    rmse = np.array(rmse)
    mae = np.array(mae)
    print(f'MSE: mean={np.mean(mse)}, std={np.std(mse)}')
    print(f'RMSE: mean={np.mean(rmse)}, std={np.std(rmse)}')
    print(f'MAE: mean={np.mean(mae)}, std={np.std(mae)}')

model_evaluation(best_mse, best_rmse, best_mae)

# check time
print()
print('[training time]')
print(f'mean : {np.mean(np.array(learning_time))}, last:{learning_time[-1]}')

MSE: mean=3.4633554940375055, std=6.785443827662659
RMSE: mean=1.2286695429096524, std=1.3977577216254937
MAE: mean=1.0350470497400062, std=1.2969288385819366

[training time]
mean : 241.47360501289367, last:407.90477871894836
